# Creation of the SWMM database
Create and export hydraulic head and runoff simulations for synthetic or real rainfalls

### Preliminaries

In [2]:
# %load_ext jupyter_black
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

sys.path.append("..")
sys.path.append("../libraries")

from libraries.utils import load_yaml, load_pickle
import libraries.utils_data as ud

load_dotenv()
data_folder = Path(os.getenv("DATA_FOLDER"))
yaml_folder = Path(os.getenv("YAML_FOLDER"))
saved_objects_folder = Path(os.getenv("SAVED_OBJECTS_FOLDER"))

Configuration information

In [2]:
yaml_name = "Tuindorp development GNN.yaml"

yaml_path = Path(yaml_folder) / yaml_name

yaml_data = load_yaml(yaml_path)

inp_path = (
    saved_objects_folder
    / "networks"
    / yaml_data["network"]
    / (yaml_data["network"] + ".inp")
)

In [3]:
type_datasets = ["training", "validation", "testing"]

rainfall_dats_directories = [
    data_folder / yaml_data["network"] / "rainfall_dats" / type_dataset
    for type_dataset in type_datasets
]

simulation_directories = [
    data_folder / yaml_data["network"] / "simulations" / type_dataset
    for type_dataset in type_datasets
]

for dir in rainfall_dats_directories + simulation_directories:
    if not dir.exists():
        dir.mkdir(parents=True)

## Synthetic rainfalls

Alternating blocks method

In [ ]:
n_synthetic_rainfalls = [
    20,
    15,
    0,
]  # The number of synthetic rainfalls for training, validation and testing

range_A = [15000, 25000]
params_B = [20, 30]
range_D = [4, 36]

c = 0
for i in range(3):

    alternating_random_rainfalls = ud.get_multiple_alt_blocks_rainfalls(
        n_synthetic_rainfalls[i],
        dt=1,
        range_A=range_A,
        params_B=params_B,
        range_D=range_D,
    )
    ud.create_datfiles(
        alternating_random_rainfalls,
        rainfall_dats_directories[i],
        identifier="synt",
        isReal=False,
        offset=c,
    )
    c += n_synthetic_rainfalls[i]

## Real rainfalls

In [ ]:
n_real_rains = [80, 15, 30]
real_rainfalls_directory = saved_objects_folder / "real_rainfalls"
pixel2 = load_pickle(real_rainfalls_directory / "events_pixel2_2014_5h.pk")

In [ ]:
rains_larger_than_ten = []
# There are some rains that have None values, these are catched with the try.
c = 0
for i in range(3):
    rains_of_type = []
    while len(rains_of_type) < n_real_rains[i] and c <= 200 * (i + 1):
        try:
            max_rain = ud.get_max_from_raindict(pixel2[c])
        except Exception as e:
            max_rain = 0
            print("error with rain #" + str(c))
        if max_rain > 10:
            rains_of_type.append(pixel2[c])
        c += 1
    rains_larger_than_ten.append(rains_of_type)

print("There are", len(rains_larger_than_ten[2]), "rains larger than 10 mm")

In [ ]:
c = 0
for i, rain_dir in enumerate(rainfall_dats_directories):
    ud.create_datfiles(
        rains_larger_than_ten[i], rain_dir, identifier="real", isReal=True, offset=c
    )
    c += n_real_rains[i]

## Run SWMM 

In [ ]:
for i in range(3):
    ud.run_SWMM(inp_path, rainfall_dats_directories[i], simulation_directories[i])

## Extract and export SWMM results to CSV

In [ ]:
for i in range(3):
    ud.extract_SWMM_results(simulation_directories[i])